# ETL

## Issues - Bronze
- The field "education_2n Cycle" has a space in the column name which should be removed
- Rows with negative values in the MntRegularProds column must be removed to avoid data inconsistencies.

## Findings

- The income field displays a broad distribution. Without additional data to assess the significance of this observation, it will remain unaltered for now.
- The fourth quartile for recency is 74 days or more, indicating potential churn.
- The minimum age of participants is 24, while the maximum is 80
- Among all categories, MntMeatProducts has the highest average spending.
- Purchases are predominantly made in-store and web.
- The second campaign has the lowest acceptance rate, whereas the fourth campaign is the most successful.
- The target (Response) shows higher average acceptance compared to previous campaigns.
- The campaigns were aimed at customers who have been with the company for between 5 to 8 years.
- Marital status: The campaigns were primarily targeted at couples (married and living together)
- Education: The focus was more on individuals with a graduation level of education.
- Spending: People tend to spend more on regular products than on gold products.

## Flags and Metrics to be added - Silver
- **Dependents**: The total number of dependents in the home can be engineered from the sum of 'Kidhome' and 'Teenhome'
- **TotalPurchases**: The total purchases can be engineered from the sum of all features containing the keyword 'Purchases'
- **RFM Scores (Recency, Frequency, Monetary)**: Using the recency, TotalPurchases and MntTotal we can engineer the score and define segments
- **AgeGroup**> 1: [0,24] 2: [24,39] 3: [40,55] 4: [56,71] 5: 71+
- **Income Group** (based on the quantile)> 1: [0, 35196] 2: [35196, 51287] 3: [51287, 68281] 4: 68281+
- **Graduation**: 2nCycle, Basic, Graduation, Master, PhD
- **Marital**: Divorced, Married, Single, Together, Widow
- **is_graduated**: 1 if has graduation, master or phD. 0 otherwise
- **is_couple**: 1 if together or married. 0 otherwise
- **has_child**: 1 if has child or teen 0 otherwise
- **is_single_parenting**: 1 if is_couple is false and has_child is true. 0 otherwise
- **is_churn**: 1 if customer present in the 4th quartil of recency (recency > 74 days), 0 otherwise), 0 otherwisegold products.

In [1]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\monic\Desktop\Projects\rauva-mkt


In [2]:
# load libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import seaborn as sns # for data visualization
import matplotlib.pyplot as plt # for plot
import missingno as msno # visualizing missing values
import warnings
%matplotlib inline
sns.set_palette('Oranges')
# turn off warnings for final notebook
warnings.filterwarnings('ignore')

In [3]:
# load dataset
input_df = pd.read_csv('./ifood_df.csv')

## BRONZE
**Data Cleaning**

####  1. Columns names standardization
Remove spaces and "_" from column names

In [4]:
input_df.columns = input_df.columns.str.replace(' ', '')
input_df.columns = input_df.columns.str.replace('_', '')

In [5]:
input_df.columns

Index(['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'ZCostContact', 'ZRevenue', 'Response',
       'Age', 'CustomerDays', 'maritalDivorced', 'maritalMarried',
       'maritalSingle', 'maritalTogether', 'maritalWidow', 'education2nCycle',
       'educationBasic', 'educationGraduation', 'educationMaster',
       'educationPhD', 'MntTotal', 'MntRegularProds', 'AcceptedCmpOverall'],
      dtype='object')

In [6]:
for i in input_df.columns:
    print(i) if ' ' in i else ""

✓ Spaces in column education_2nCycle removed

####  2. Remove rows with negative values
Remove rows with negative values on MntRegularProds

In [7]:
input_df[input_df['MntRegularProds'] < 0]

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,maritalTogether,maritalWidow,education2nCycle,educationBasic,educationGraduation,educationMaster,educationPhD,MntTotal,MntRegularProds,AcceptedCmpOverall
1307,6560.0,0,0,2,67,11,26,4,3,262,...,0,0,0,0,0,1,0,111,-151,0
1780,7144.0,0,2,92,81,4,33,5,2,291,...,0,0,0,0,0,0,1,125,-166,0
1949,4428.0,0,1,0,16,4,12,2,4,321,...,0,0,0,0,1,0,0,38,-283,0


In [8]:
# Count the number of negative values in 'MntRegularProds'
num_negative_values = (input_df['MntRegularProds'] < 0).sum()

# Select the distinct negative values from 'MntRegularProds'
distinct_negative_values = input_df.loc[input_df['MntRegularProds'] < 0, 'MntRegularProds'].unique()

num_negative_values, distinct_negative_values

(3, array([-151, -166, -283], dtype=int64))

In [9]:
# remove values
cleaned_df = input_df[input_df['MntRegularProds'] >= 0]
cleaned_df.shape

(2202, 39)

✓ 3 Rows removed

## SILVER
**Add columns**

#### Dependents

In [10]:
cleaned_df["Dependents"] = cleaned_df["Kidhome"] + cleaned_df["Teenhome"] 

In [11]:
cleaned_df[['Kidhome', 'Teenhome', 'Dependents']].head()

,Kidhome,Teenhome,Dependents
0,0,0,0
1,1,1,2
2,0,0,0
3,1,0,1
4,1,0,1


#### TotalPurchases

In [12]:
cleaned_df["TotalPurchases"] = (cleaned_df["NumDealsPurchases"] 
                                + cleaned_df["NumWebPurchases"]
                                + cleaned_df["NumCatalogPurchases"] 
                                + cleaned_df["NumStorePurchases"]
                               )

In [13]:
cleaned_df[['NumDealsPurchases','NumWebPurchases','NumCatalogPurchases', 'NumStorePurchases', 'TotalPurchases']].head()

,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,TotalPurchases
0,3,8,10,4,25
1,2,1,1,2,6
2,1,8,2,10,21
3,2,2,0,4,8
4,5,5,3,6,19


#### RFM Scores (Recency, Frequency, Monetary)
With RFM scores or segments, we can identify high-value customers (e.g., those with scores of '444'), new or at-risk customers (e.g., '411' or '111'), and tailor the marketing strategies accordingly.

In [14]:
# Scoring Recency: Here, a lower recency is better, so we invert the scores
cleaned_df['R_Score'] = pd.qcut(cleaned_df['Recency'], 4, labels=[4, 3, 2, 1])

# Scoring Frequency: Higher frequency is better
cleaned_df['F_Score'] = pd.qcut(cleaned_df['TotalPurchases'], 4, labels=[1, 2, 3, 4])

# Scoring Monetary: Higher monetary value is better
cleaned_df['M_Score'] = pd.qcut(cleaned_df['MntTotal'], 4, labels=[1, 2, 3, 4])

# Concatenate RFM scores
cleaned_df['RFM_Segment'] = cleaned_df['R_Score'].astype(str) + cleaned_df['F_Score'].astype(str) + cleaned_df['M_Score'].astype(str)

# Calculate an overall RFM score (optional)
cleaned_df['RFM_Score'] = cleaned_df[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

#### AgeGroup
1: 0-30 #Emerging Adults
2: 31-45 #Established Adults 
3: 46-60 #Experienced Adults
4: 61-74 #Senior Adults
5: 75+ #Wisdom Elders

In [19]:
# option by divind in 5 groups by quantile
# cleaned_df['AgeGroup'] = pd.qcut(cleaned_df['Age'], 5, labels=[1, 2, 3, 4, 5])
# age_group_ranges = cleaned_df.groupby('AgeGroup')['Age'].agg(['min', 'max'])
# print(age_group_ranges)

In [20]:
# cleaned_df[["Age", "AgeGroup"]].head()

In [21]:
# option by divind in 5 groups by Age - choosen one, since from an analytics perpective in terms of customer behaviour makes more sense
cleaned_df.loc[cleaned_df['Age'].between(0, 30, inclusive='both'), 'AgeGroup'] = 1 # Youth
cleaned_df.loc[cleaned_df['Age'].between(31, 45, inclusive='both'), 'AgeGroup'] = 2 # Adult
cleaned_df.loc[cleaned_df['Age'].between(46, 60, inclusive='both'), 'AgeGroup'] = 3 # Old Adult
cleaned_df.loc[cleaned_df['Age'].between(61, 74, inclusive='both'), 'AgeGroup'] = 4 # Old
cleaned_df.loc[cleaned_df['Age'].gt(75), 'AgeGroup'] = 5 #Eldery

In [22]:
age_group_ranges = cleaned_df.groupby('AgeGroup')['Age'].agg(['min', 'max'])
print(age_group_ranges)

          min  max
AgeGroup          
1.0        24   30
2.0        31   45
3.0        46   60
4.0        61   74
5.0        76   80


#### Spender Group
The following segmentation utilizes  descriptive analysis to the data, like mean, max, minimum to define the groups. Adjustments should be made to these thresholds to better align with specific business needs or local economic conditions!
- Low Spenders: < 500
- Medium Spenders: Between 25% and 50% percentiles 500 to 1000
- High Spenders: Between 50% and 75% percentiles 1000 to 2000
- Very High Spenders: Above the 75% percentile > 964

In [49]:
def assign_income_group(row):
    if row['MntTotal'] < 500:
        return 'Low'
    elif row['MntTotal'] <= 1000:
        return 'Medium'
    elif row['MntTotal'] <= 2000:
        return 'High'
    else:
        return 'VeryHigh'

cleaned_df['SpenderGroup'] = cleaned_df.apply(assign_income_group, axis=1)

#### Income Group
The following segmentation utilizes natural breaks in the income distribution, ensuring each group represents a distinct economic stratum. Adjustments should be made to these thresholds to better align with specific business needs or local economic conditions!
- Low Income: Below the 25% percentile (< 35196)
- Middle Income: Between the 25% and 75% percentiles (35196 to 68281)
- High Income: Above the 75% percentile (> 68,281)1)

In [24]:
def assign_income_group(row):
    if row['Income'] < 35196:
        return 'Low'
    elif row['Income'] <= 68281:
        return 'Middle'
    else:
        return 'High'

cleaned_df['IncomeGroup'] = cleaned_df.apply(assign_income_group, axis=1)

In [25]:
cleaned_df[['Income','IncomeGroup']]

,Income,IncomeGroup
0,58138.0,Middle
1,46344.0,Middle
2,71613.0,High
3,26646.0,Low
4,58293.0,Middle
...,...,...
2200,61223.0,Middle
2201,64014.0,Middle
2202,56981.0,Middle
2203,69245.0,High


#### Education

In [27]:
def get_education(row):
    if row['educationPhD'] == 1:
        return 'PhD'
    elif row['educationMaster'] == 1:
        return 'Master'
    elif row['educationGraduation'] == 1:
        return 'Graduation'
    elif row['education2nCycle'] == 1:
        return '2nCycle'
    elif row['educationBasic'] == 1:
        return 'Basic'
    else:
        return 'Unknown'

cleaned_df['Education'] = cleaned_df.apply(get_education, axis=1)

In [28]:
cleaned_df[['educationPhD','educationMaster','educationGraduation','education2nCycle','educationBasic','Education']]

,educationPhD,educationMaster,educationGraduation,education2nCycle,educationBasic,Education
0,0,0,1,0,0,Graduation
1,0,0,1,0,0,Graduation
2,0,0,1,0,0,Graduation
3,0,0,1,0,0,Graduation
4,1,0,0,0,0,PhD
...,...,...,...,...,...,...
2200,0,0,1,0,0,Graduation
2201,1,0,0,0,0,PhD
2202,0,0,1,0,0,Graduation
2203,0,1,0,0,0,Master


In [29]:
unknown_education_df = cleaned_df[cleaned_df['Education'] == 'Unknown']
unknown_education_df

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,Dependents,TotalPurchases,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score,AgeGroup,IncomeGroup,Education


#### MaritalStatus

In [31]:
def get_marital(row):
    if row['maritalDivorced'] == 1:
        return 'Divorced'
    elif row['maritalMarried'] == 1:
        return 'Married'
    elif row['maritalSingle'] == 1:
        return 'Single'
    elif row['maritalTogether'] == 1:
        return 'Together'
    elif row['maritalWidow'] == 1:
        return 'Widow'
    else:
        return 'Unknown'

cleaned_df['MaritalStatus'] = cleaned_df.apply(get_marital, axis=1)

In [32]:
cleaned_df[['maritalDivorced','maritalMarried','maritalSingle','maritalTogether','maritalWidow','MaritalStatus']]

,maritalDivorced,maritalMarried,maritalSingle,maritalTogether,maritalWidow,MaritalStatus
0,0,0,1,0,0,Single
1,0,0,1,0,0,Single
2,0,0,0,1,0,Together
3,0,0,0,1,0,Together
4,0,1,0,0,0,Married
...,...,...,...,...,...,...
2200,0,1,0,0,0,Married
2201,0,0,0,1,0,Together
2202,1,0,0,0,0,Divorced
2203,0,0,0,1,0,Together


In [33]:
unknown_marital_df = cleaned_df[cleaned_df['MaritalStatus'] == 'Unknown']
unknown_marital_df

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,TotalPurchases,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score,AgeGroup,IncomeGroup,Education,MaritalStatus


#### PreferredChannel

In [35]:
# Creating a subset DataFrame with relevant columns
purchase_channels = cleaned_df[['NumDealsPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebPurchases']]

# Identifying the preferred channel for each customer
cleaned_df['PreferredChannel'] = purchase_channels.idxmax(axis=1)

# Optional: Rename the resulting labels to more user-friendly names
channel_mapping = {
    'NumDealsPurchases': 'Deals',
    'NumCatalogPurchases': 'Catalog',
    'NumStorePurchases': 'Store',
    'NumWebPurchases': 'Web'
}
cleaned_df['PreferredChannel'] = cleaned_df['PreferredChannel'].map(channel_mapping)

#### Flags (is_graduated, is_couple, has_child, is_single_parenting, is_churn)

In [37]:
# is_graduated flag
cleaned_df['isGraduated'] = cleaned_df[['educationMaster', 'educationPhD', 'educationGraduation']].max(axis=1)
# is_couple flag
cleaned_df['isCouple'] = cleaned_df[['maritalMarried', 'maritalTogether']].max(axis=1) 
# has_child flag
cleaned_df['hasChild'] = (cleaned_df['Dependents'] > 0).astype(int)
# is_single_parenting flag
cleaned_df['isSingleParenting'] = ((cleaned_df['isCouple'] == 0) & (cleaned_df['hasChild'] == 1)).astype(int)
# is_churn flag
cleaned_df['isChurn'] = (cleaned_df['Recency'] > 74).astype(int)  

##### Test

In [38]:
unique_education_graduated = cleaned_df[['Education', 'isGraduated']].drop_duplicates()
print(unique_education_graduated)

     Education  isGraduated
0   Graduation            1
4          PhD            1
5       Master            1
10       Basic            0
18     2nCycle            0


In [39]:
unique_MaritalStatus = cleaned_df[['MaritalStatus', 'isCouple']].drop_duplicates()
print(unique_MaritalStatus)

   MaritalStatus  isCouple
0         Single         0
2       Together         1
4        Married         1
6       Divorced         0
36         Widow         0


In [40]:
unique_has_child = cleaned_df[['hasChild', 'Dependents']].drop_duplicates()
print(unique_has_child)

    hasChild  Dependents
0          0           0
1          1           2
3          1           1
64         1           3


In [41]:
unique_is_single_parenting = cleaned_df[['isSingleParenting', 'isCouple', 'hasChild']].drop_duplicates()
print(unique_is_single_parenting)

   isSingleParenting  isCouple  hasChild
0                  0         0         0
1                  1         0         1
2                  0         1         0
3                  0         1         1


In [42]:
churn_df = cleaned_df[cleaned_df['isChurn'] == 1]
churn_df[["Recency", "isChurn"]]

,Recency,isChurn
4,94,1
11,82,1
17,91,1
18,86,1
24,89,1
...,...,...
2187,81,1
2193,87,1
2196,82,1
2199,81,1


In [43]:
cleaned_df.columns = cleaned_df.columns.str.replace(' ', '')
cleaned_df.columns = cleaned_df.columns.str.replace('_', '')

In [44]:
cleaned_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Income,2202.0,51684.190282,20658.620341,1730.0,35265.0,51342.0,68307.25,113734.0
Kidhome,2202.0,0.442779,0.537250,0.0,0.0,0.0,1.00,2.0
Teenhome,2202.0,0.505904,0.543611,0.0,0.0,0.0,1.00,2.0
Recency,2202.0,49.033152,28.901080,0.0,24.0,49.0,74.00,99.0
MntWines,2202.0,306.507266,337.594355,0.0,24.0,178.5,507.00,1493.0
MntFruits,2202.0,26.430518,39.804496,0.0,2.0,8.0,33.00,199.0
MntMeatProducts,2202.0,165.504995,217.869804,0.0,16.0,68.0,232.00,1725.0
MntFishProducts,2202.0,37.802906,54.847508,0.0,3.0,12.0,50.00,259.0
MntSweetProducts,2202.0,27.161217,41.148824,0.0,1.0,8.0,34.00,262.0
MntGoldProds,2202.0,43.720254,50.951321,0.0,9.0,25.0,56.00,249.0


In [45]:
cleaned_df.columns

Index(['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'ZCostContact', 'ZRevenue', 'Response',
       'Age', 'CustomerDays', 'maritalDivorced', 'maritalMarried',
       'maritalSingle', 'maritalTogether', 'maritalWidow', 'education2nCycle',
       'educationBasic', 'educationGraduation', 'educationMaster',
       'educationPhD', 'MntTotal', 'MntRegularProds', 'AcceptedCmpOverall',
       'Dependents', 'TotalPurchases', 'RScore', 'FScore', 'MScore',
       'RFMSegment', 'RFMScore', 'AgeGroup', 'IncomeGroup', 'Education',
       'MaritalStatus', 'PreferredChannel', 'isGraduated', 'isCouple',
       'hasChild', 'isSingleParenting', 'isChurn'],
      dtype='object')

In [50]:
cleaned_df.to_csv('ifood_silver.csv', index=False)